In [2]:

# good references
# https://www.tensorflow.org/tutorials/load_data/pandas_dataframe
# https://www.tensorflow.org/tutorials/load_data/csv

import os

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

from keras import layers
from keras import Input

tf.__version__

2023-08-27 23:01:34.923427: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-27 23:01:34.950541: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-27 23:01:34.951868: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-27 23:01:35.584812: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


'2.13.0'

In [3]:
# TODO there is a numpy util to get files from within tgz's and zips
##!curl -s 'https://storage.googleapis.com/0x19f.com/media/kaggle-mlb-pitch-data-2015-2018.tgz' | tar xz

BASEBALL_FILES_BASE = os.getcwd() + "/kaggle-mlb-pitch-data-2015-2018/"
BASEBALL_FILES_BASE = BASEBALL_FILES_BASE + "2019_"  # just look at 2019 to speed things up


pitches = pd.read_csv(BASEBALL_FILES_BASE + 'pitches.csv')
atbats = pd.read_csv(BASEBALL_FILES_BASE + 'atbats.csv')

df = pd.merge(pitches, atbats, how='inner', on='ab_id')

df = df[[
    # situation stuff
    "ab_id", "inning", "top", "outs", "on_1b", "on_2b", "on_3b",

    # matchup
    "pitcher_id", "p_throws", "batter_id", "stand",

    # specific pitch state
    "pitch_num", "b_count", "s_count",
    # TODO need to add running total of pitches for the current pitcher, and maybe strike percentage
    # TODO add the previous n pitches before and their code outcome
    # 
    
    # this is the label we're trying to predict
    "code",
    "pitch_type",
    # TODO also make the label have the location. not just FF (fourseam fastball, but also high inside)
]]

# remove anything without values, hard to have defaults
# df = df.dropna(subset=['pitch_type', 'code'])
df = df.dropna()

## this is our current target label TODO make this categorical 
df["is_fastball"] = np.where(df["pitch_type"].isin(["FC","FF","FT",]), 1, 0)

# get appropriate types. 
# TODO prob can use a schema on the CSV read to get only the columns we're after, alias them to be more
#      useful, and cast to the right type
# TODO consider to_ordinal for cumulative things maybe? inning, outs, pitch_num, b_count, s_count

# TODO there is a numpy util to get files from within tgz's and zips
##!curl -s 'https://storage.googleapis.com/0x19f.com/media/kaggle-mlb-pitch-data-2015-2018.tgz' | tar xz

BASEBALL_FILES_BASE = os.getcwd() + "/kaggle-mlb-pitch-data-2015-2018/"
BASEBALL_FILES_BASE = BASEBALL_FILES_BASE + "2019_"  # just look at 2019 to speed things up


pitches = pd.read_csv(BASEBALL_FILES_BASE + 'pitches.csv')
atbats = pd.read_csv(BASEBALL_FILES_BASE + 'atbats.csv')

df = pd.merge(pitches, atbats, how='inner', on='ab_id')

df = df[[
    # situation stuff
    "ab_id", "inning", "top", "outs", "on_1b", "on_2b", "on_3b",

    # matchup
    "pitcher_id", "p_throws", "batter_id", "stand",

    # specific pitch state
    "pitch_num", "b_count", "s_count",
    # TODO p0: multi hot encode the count
    # TODO need to add running total of pitches for the current pitcher, and maybe strike percentage
    # TODO add the previous n pitches before and their code outcome
    # 
    
    # this is the label we're trying to predict
    "code",
    "pitch_type",
    # TODO also make the label have the location. not just FF (fourseam fastball, but also high inside)
]]

# remove anything without values, hard to have defaults
# df = df.dropna(subset=['pitch_type', 'code'])
df = df.dropna()

## this is our current target label TODO make this categorical 
df["is_fastball"] = np.where(df["pitch_type"].isin(["FC","FF","FT",]), 1, 0)

# get appropriate types. 
# TODO prob can use a schema on the CSV read to get only the columns we're after, alias them to be more
#      useful, and cast to the right type
# TODO consider to_ordinal for cumulative things maybe? inning, outs, pitch_num, b_count, s_count


# TODO consider using ints and normalizing them instead of floats for innnigs etc

df = df.astype({
    # string features
    'p_throws': 'string',
    'stand': 'string',

    # small card numeric features
    'inning': 'int8',
    'top': 'int8',
    'outs': 'int8',
    'on_1b': 'int8',
    'on_2b': 'int8',
    'on_3b': 'int8',
    'pitch_num': 'int8',
    'b_count': 'int8',
    's_count': 'int8',

    # player identifiers
    'pitcher_id': 'int64', 
    'batter_id': 'int64',
  })


# drop ab_id now that join has occured, not a useful feature
df = df.drop(['ab_id', "code","pitch_type"], axis=1)

# TODO remove this truncation when solid
df = df.truncate(after=100*1000)


pitch_features = df.copy()
pitch_labels = pitch_features.pop('is_fastball')

print(pitch_features.info())
print(pitch_features.shape)

pitch_features.head()


<class 'pandas.core.frame.DataFrame'>
Index: 98972 entries, 0 to 100000
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   inning      98972 non-null  int8  
 1   top         98972 non-null  int8  
 2   outs        98972 non-null  int8  
 3   on_1b       98972 non-null  int8  
 4   on_2b       98972 non-null  int8  
 5   on_3b       98972 non-null  int8  
 6   pitcher_id  98972 non-null  int64 
 7   p_throws    98972 non-null  string
 8   batter_id   98972 non-null  int64 
 9   stand       98972 non-null  string
 10  pitch_num   98972 non-null  int8  
 11  b_count     98972 non-null  int8  
 12  s_count     98972 non-null  int8  
dtypes: int64(2), int8(9), string(2)
memory usage: 4.6 MB
None
(98972, 13)


,inning,top,outs,on_1b,on_2b,on_3b,pitcher_id,p_throws,batter_id,stand,pitch_num,b_count,s_count
0,1,1,0,0,0,0,571666,R,594777,L,1,0,0
1,1,1,1,0,0,0,571666,R,545361,R,1,0,0
2,1,1,1,0,0,0,571666,R,545361,R,2,0,0
3,1,1,1,0,0,0,571666,R,545361,R,3,0,1
4,1,1,1,0,0,0,571666,R,545361,R,4,1,1


In [4]:


feature_inputs, processed_inputs = {}, []

for name, col in pitch_features.items():
    print("processing: %s ..." % name)
    col_type_str = str(col.dtype)
    tf_type_for_col = tf.as_dtype(col_type_str)

    # first create the feature's input and collect into feature_inputs
    col_input = tf.keras.Input(shape=(1,), name=name, dtype=tf_type_for_col)
    feature_inputs[name] = col_input

    # next, normalize the feature values and collect them in feature_normalizers
    if col_type_str == "string":       
        feature_lookup = layers.StringLookup(vocabulary=np.unique(pitch_features[name]))
        one_hot = layers.CategoryEncoding(num_tokens=feature_lookup.vocabulary_size(), output_mode="one_hot")

        col_values_from_lookup = feature_lookup(col_input)
        normalizer = one_hot(col_values_from_lookup)

    else:  # TODO should prob do a better job of figuring out numerics
        norm = layers.Normalization()
        norm.adapt(col)
        normalizer = norm(col_input)
        
    # here we append the normalizer from the respective types
    processed_inputs.append(normalizer)

feature_inputs, len(processed_inputs)


processing: inning ...


2023-08-27 23:01:53.890011: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


processing: top ...
processing: outs ...
processing: on_1b ...
processing: on_2b ...
processing: on_3b ...
processing: pitcher_id ...
processing: p_throws ...
processing: batter_id ...
processing: stand ...
processing: pitch_num ...
processing: b_count ...
processing: s_count ...


({'inning': <KerasTensor: shape=(None, 1) dtype=int8 (created by layer 'inning')>,
  'top': <KerasTensor: shape=(None, 1) dtype=int8 (created by layer 'top')>,
  'outs': <KerasTensor: shape=(None, 1) dtype=int8 (created by layer 'outs')>,
  'on_1b': <KerasTensor: shape=(None, 1) dtype=int8 (created by layer 'on_1b')>,
  'on_2b': <KerasTensor: shape=(None, 1) dtype=int8 (created by layer 'on_2b')>,
  'on_3b': <KerasTensor: shape=(None, 1) dtype=int8 (created by layer 'on_3b')>,
  'pitcher_id': <KerasTensor: shape=(None, 1) dtype=int64 (created by layer 'pitcher_id')>,
  'p_throws': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'p_throws')>,
  'batter_id': <KerasTensor: shape=(None, 1) dtype=int64 (created by layer 'batter_id')>,
  'stand': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'stand')>,
  'pitch_num': <KerasTensor: shape=(None, 1) dtype=int8 (created by layer 'pitch_num')>,
  'b_count': <KerasTensor: shape=(None, 1) dtype=int8 (created by layer 'b_

In [5]:
all_processed_inputs = layers.Concatenate()(processed_inputs)
pitch_preprocessing = tf.keras.Model(feature_inputs, all_processed_inputs)

# tf.keras.utils.plot_model(model = pitch_preprocessing , rankdir="LR", dpi=72, show_shapes=True)

In [7]:

seq_model = tf.keras.Sequential([
    layers.Dense(128, activation="relu"),
    layers.Dense(64, activation="relu"),
    layers.Dense(1, activation="relu"),
  ])


preprossesed_inputs = pitch_preprocessing(feature_inputs)
result = seq_model(preprossesed_inputs)
pitch_model = tf.keras.Model(feature_inputs, result)

pitch_model.compile("adam", "binary_crossentropy", run_eagerly=True, metrics=["accuracy"])

pitch_features_dict = {
  name: np.array(value) for name, value in pitch_features.items()
}
pitch_model.fit(x=pitch_features_dict, y=pitch_labels, epochs=500, batch_size=20*1000)



Epoch 1/50
3/3 [==============================] - 0s 63ms/step - loss: 0.9412 - accuracy: 0.5105
Epoch 2/50
3/3 [==============================] - 0s 71ms/step - loss: 0.7434 - accuracy: 0.5169
Epoch 3/50
3/3 [==============================] - 0s 68ms/step - loss: 0.7209 - accuracy: 0.5181
Epoch 4/50
3/3 [==============================] - 0s 75ms/step - loss: 0.7153 - accuracy: 0.5234
Epoch 5/50
3/3 [==============================] - 0s 73ms/step - loss: 0.7111 - accuracy: 0.5293
Epoch 6/50
3/3 [==============================] - 0s 75ms/step - loss: 0.7044 - accuracy: 0.5285
Epoch 7/50
3/3 [==============================] - 0s 56ms/step - loss: 0.6997 - accuracy: 0.5276
Epoch 8/50
3/3 [==============================] - 0s 67ms/step - loss: 0.6966 - accuracy: 0.5285
Epoch 9/50
3/3 [==============================] - 0s 62ms/step - loss: 0.6947 - accuracy: 0.5305
Epoch 10/50
3/3 [==============================] - 0s 61ms/step - loss: 0.6939 - accuracy: 0.5318
Epoch 11/50
3/3 [============